In [108]:
import pandas as pd
import numpy as np
from statistics import geometric_mean

import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [109]:
# import result sets
df_ml = pd.read_csv("ml.csv")
df_ml = df_ml.drop(['Unnamed: 0', 'GAP W/ CUTS', 
                   'GAP W/O CUTS'], axis = 1)
df_ml.rename(columns={'INITIAL LP':'INITIAL LP W/O CUTS', 'PRESOLVING TIME':'PRESOLVING TIME W/O CUTS'}, inplace=True)

df = pd.read_csv("results_1_new.csv")

analysis_ml = df_ml.merge(df, on = ['NAME', 'RANDOM SEED'], how = 'left')
analysis_ml = analysis_ml.drop(['BEST SOLUTION', 'BEST PRIMAL BOUND', 'GAP', 'No. OF CUTS GENERATED', 'No. OF CUTS APPLIED',
                               'No. OF CUTS PRESENT IN LP AT THE END', 'No. OF NODES', 'RANDOM SEED', 'CUT SETTING',
                               'STATUS', 'END', 'SOLUTION TIME'], axis = 1)
analysis_ml.rename(columns={'INITIAL LP':'INITIAL LP W/ CUTS', 'PRESOLVING TIME':'PRESOLVING TIME W/ CUTS'}, inplace=True)
analysis_ml['CUTS OFF_FIRST ROUND'] = analysis_ml['TIME W/O CUTS'] + analysis_ml['TIME FIRST ROUND']
analysis_ml['CUTS OFF_ROOT END'] = analysis_ml['TIME W/O CUTS'] + analysis_ml['TIME ROOT END']
analysis_ml = analysis_ml.drop(['INITIAL LP TIME', 'TIME W/O CUTS', 'CUTS OFF_FIRST ROUND'], axis = 1)

analysis_ml = analysis_ml[['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 'PRESOLVING TIME W/ CUTS', 'PRESOLVING TIME W/O CUTS',
                            'TIME FIRST ROUND', 'TIME ROOT END','NODES W/ CUTS', 'NODES W/O CUTS', 'INITIAL LP W/ CUTS', 
                           'INITIAL LP W/O CUTS', 'FIRST ROUND CUT',
                           'ROUND OF CUTS', 'OBJECTIVE VALUE ROOT NODE', 'vars', 'bin_vars', 'int_vars', 'cont_vars', 
                           'constr', 'linsetpart_constr', 'linsetpack_constr', 'linsetcov_constr', 'lincard_constr', 
                           'lineqknap_constr', 'linbinpack_constr', 'linknaps_constr', 'linintknap_constr', 'linmixbin_constr',
                           'nonzeroes', 'obj_dyn', 'dynamism_mean', 'presol_vars','presol_int_vars', 'presol_constr']]

In [110]:
analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,TIME ROOT END,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,ROUND OF CUTS,OBJECTIVE VALUE ROOT NODE,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,11.088517,15.713885,0.067284,0.062201,1.426593,3.501355,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
1,10teams.mps.gz,11.186637,15.838753,0.057992,0.060087,1.378128,3.394305,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
2,10teams.mps.gz,11.044666,15.637111,0.055842,0.058780,1.350495,3.305622,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
3,10teams.mps.gz,10.835616,15.641646,0.052585,0.059116,1.293845,3.174460,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
4,10teams.mps.gz,10.833424,15.666084,0.053544,0.058801,1.301532,3.188138,30,276,20.0,20.0,917.0,13,924.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,wachplan.mps.gz,1427.930347,1900.337469,1.165874,1.552340,2.689512,98.732839,67520,98056,-24.0,-24.0,-9.0,12,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
2126,wachplan.mps.gz,1444.662760,1901.090928,1.031269,1.914422,2.536143,96.248496,67520,98056,-24.0,-24.0,-9.0,12,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
2127,wachplan.mps.gz,1431.659624,1833.795217,1.088057,1.507830,2.595248,96.643203,67520,98056,-24.0,-24.0,-9.0,12,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
2128,wachplan.mps.gz,1433.459125,1856.035662,1.125112,1.547767,2.649715,97.447706,67520,98056,-24.0,-24.0,-9.0,12,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025


In [111]:
# averaging over each instance for reduced data set
analysis_ml = analysis_ml.groupby('NAME').mean()
analysis_ml = analysis_ml.reset_index()

In [112]:
# Time Label
time_label_1 = []

for index, row in analysis_ml.iterrows():
    if row['TIME W/ CUTS'] > row['CUTS OFF_ROOT END']:
        time_label_1.append(0)
    elif row['TIME W/ CUTS'] <= row['CUTS OFF_ROOT END']:
        time_label_1.append(1)
    
analysis_ml['Time Label'] = time_label_1

analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,TIME ROOT END,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,ROUND OF CUTS,OBJECTIVE VALUE ROOT NODE,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,10teams.mps.gz,10.997772,15.699496,0.057449,0.059797,1.350119,3.312776,30.0,276.0,2.000000e+01,2.000000e+01,9.170000e+02,13.0,9.240000e+02,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.000000,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108,1
1,22433.mps.gz,2.535930,3.478681,0.046082,0.025466,0.172883,2.828100,1.0,5.0,2.124053e+04,2.124053e+04,2.127800e+04,58.0,2.147114e+04,6.061457,0.538462,0.000000,0.461538,5.288267,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267,1
2,23588.mps.gz,5.527370,4.372027,0.027812,0.031946,0.140688,1.972278,576.0,981.0,7.649866e+03,7.649866e+03,7.660000e+03,34.0,7.765000e+03,5.908083,0.627717,0.000000,0.372283,4.919981,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981,0
3,30_70_45_05_100.mps.gz,282.653795,402.324263,1.986556,1.625184,18.567687,162.244480,46.0,201.0,-5.336000e+03,-5.336000e+03,8.500000e+00,13.0,9.000000e+00,9.284705,0.999907,0.000000,0.000093,9.396820,0.000000,0.000000,0.004066,0.000000,0.0,0.0,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654,1
4,30_70_45_095_100.mps.gz,743.997969,246.544417,1.994498,2.143665,19.518728,190.829956,201.0,1.0,-6.132000e+03,-6.132000e+03,3.000000e+00,12.0,3.000000e+00,9.303466,0.999909,0.000000,0.000091,9.435562,0.000000,0.000000,0.001437,0.000000,0.0,0.0,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,uct-subprob.mps.gz,3212.414116,18113.345814,0.121225,0.167001,0.360338,113.335718,75300.0,2159025.8,-2.000000e+00,-2.000000e+00,2.440000e+02,52.0,2.757333e+02,7.721349,0.167996,0.000000,0.832004,7.587311,0.000000,0.003041,0.000000,0.000000,0.0,0.0,0.0,0.0,0.874303,0.002280,0.90309,0.000000,7.703459,0.000000,7.541683,1
422,umts.mps.gz,1079.840992,11640.938871,0.179507,0.230250,0.892776,36.086503,144227.0,3106370.0,1.460426e+06,1.460426e+06,2.912957e+07,30.0,2.914217e+07,7.988543,0.950797,0.024432,0.024771,8.404024,0.033595,0.000000,0.403359,0.002688,0.0,0.0,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152,1
423,unitcal_7.mps.gz,940.799926,10890.250070,6.937303,6.065447,64.953906,326.517054,705.0,196989.0,1.028415e+06,1.028415e+06,1.946934e+07,86.0,1.960908e+07,10.156384,0.110891,0.000000,0.889109,10.798330,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349,1
424,var-smallemery-m6j6.mps.gz,7171.038360,10328.248584,8.873553,8.342061,79.559820,82.023799,685889.0,836578.0,-1.697219e+03,-1.697219e+03,-1.549250e+02,1.0,-1.549250e+02,8.631949,0.999643,0.000000,0.000357,9.504203,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321,1


In [113]:
# Splitting features and label
X = analysis_ml.iloc[0:, 0:-1]
y = analysis_ml.iloc[0:, -1]

In [114]:
# Splitting training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
X_train = X_train.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 
                       'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)
cols = X_train.copy()
X_test = X_test.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 
                     'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
cols

(340, 29) (340,)
(86, 29) (86,)


,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,TIME ROOT END,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,ROUND OF CUTS,OBJECTIVE VALUE ROOT NODE,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
255,0.045498,0.017984,0.027525,2829.124553,8.877000e+03,8.877000e+03,8877.000000,29.0,1.059800e+04,5.278115,0.933673,0.000000,0.066327,5.214936,0.152174,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.847826,0.023126,3.37675,0.971717,5.278115,0.000000,5.214936
313,16.042202,22.825316,16.406966,16.410093,-4.390700e+04,-4.390700e+04,-3719.000000,1.0,-3.719000e+03,10.053114,0.999828,0.000172,0.000000,10.743415,0.000000,0.000086,0.000000,0.000259,0.0,0.0,0.04896,0.000000,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173,10.740995
229,68.027085,33.449583,556.933012,578.506055,2.282219e-03,2.282219e-03,0.002282,111.0,5.451422e-03,11.449410,0.999361,0.000000,0.000639,12.125811,0.000336,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000623,0.000034,0.00000,1.150550,11.449410,0.000000,12.125811
5,1.683165,2.150924,13.898364,69.983411,-5.967000e+03,-5.967000e+03,11.500000,11.0,1.200000e+01,9.304741,0.999909,0.000000,0.000091,9.431161,0.000000,0.000000,0.001524,0.000000,0.0,0.0,0.00000,0.000000,0.160532,0.000338,3.77706,0.000000,9.303649,0.000000,9.430359
425,1.102859,1.619734,2.617340,97.206743,-2.400000e+01,-2.400000e+01,-9.000000,12.0,-9.000000e+00,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.000000,0.144237,0.0,0.0,0.00000,0.000000,0.018030,0.017120,0.00000,0.000000,7.741534,0.000434,6.582025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,26.047749,40.978283,0.000000,0.000000,-2.000000e+05,-2.000000e+05,0.000000,0.0,-1.524333e+03,10.315597,0.331126,0.000000,0.668874,10.613934,0.004914,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.493882,0.001717,0.00000,0.642555,10.315564,0.000000,10.328690
192,7.056363,6.946319,0.000000,0.000000,-3.689729e+08,-3.689729e+08,0.000000,0.0,-3.689729e+08,8.464003,0.461928,0.000000,0.538072,8.095599,0.000000,0.111280,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.666768,0.036965,7.77789,1.758950,8.464003,0.000000,8.095599
117,0.039291,0.034147,0.067639,5.569328,2.606000e+03,2.606000e+03,3900.698555,40.0,3.936000e+03,6.555357,0.250356,0.005690,0.743954,5.765191,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.987461,0.009230,0.00000,1.003350,6.481577,0.006126,5.736572
47,0.028787,0.030815,0.052751,0.347415,3.600000e+01,3.600000e+01,487.000000,26.0,4.870000e+02,5.446737,0.987069,0.004310,0.008621,4.077537,0.288136,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.016949,0.559322,0.320500,0.00000,1.311860,5.438079,0.000000,4.060443


In [115]:
# Scaling
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [116]:
# Feature Selection
cols = cols.iloc[0:, 0:]

bestfeatures = SelectKBest(score_func=f_classif, k = 'all')
fit = bestfeatures.fit(X_train,y_train)
scores = pd.DataFrame(fit.scores_)
columns = pd.DataFrame(cols.columns)
featureScores = pd.concat([columns,scores],axis=1)
featureScores
featureScores.columns = ['Features','Score']  #naming the dataframe columns
print(featureScores.nlargest(29,'Score')) #print best features
data=featureScores.nlargest(29,'Score')

                     Features     Score
7               ROUND OF CUTS  9.068150
23                  nonzeroes  8.956953
3               TIME ROOT END  5.407416
8   OBJECTIVE VALUE ROOT NODE  2.438572
4          INITIAL LP W/ CUTS  2.437107
5         INITIAL LP W/O CUTS  2.437107
25              dynamism_mean  1.904837
21          linintknap_constr  1.891313
27            presol_int_vars  1.445125
26                presol_vars  1.406785
9                        vars  0.973086
13                     constr  0.872513
15          linsetpack_constr  0.809836
18           lineqknap_constr  0.543249
28              presol_constr  0.465052
11                   int_vars  0.460934
6             FIRST ROUND CUT  0.415949
17             lincard_constr  0.360940
22           linmixbin_constr  0.305354
2            TIME FIRST ROUND  0.296398
14          linsetpart_constr  0.207423
19          linbinpack_constr  0.165693
20            linknaps_constr  0.109086
12                  cont_vars  0.085881


In [117]:
# Dropping bottom 7 features
X = analysis_ml.drop(['linknaps_constr', 'cont_vars', 'obj_dyn', 'PRESOLVING TIME W/ CUTS',
                      'linsetcov_constr', 'PRESOLVING TIME W/O CUTS', 'bin_vars',
                      'Time Label'], axis = 1)
y = analysis_ml.iloc[0:, -1]

In [118]:
# Splitting again
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

test_df = X_test.copy()
train_df = X_train.copy()

X_train = X_train.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 'NODES W/ CUTS', 
                        'NODES W/O CUTS'], axis = 1)
X_test = X_test.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END',
                     'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(340, 22) (340,)
(86, 22) (86,)


In [119]:
# scaling
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [120]:
# grid search c
model_params={'svm':{'model': SVC(kernel = 'rbf', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 'gamma': [1e-2, 1e-3, 1e-4]
                             }
                    },
              'rfm':{'model': RandomForestClassifier(random_state = 0), 
                    'params':{'n_estimators':[100, 200, 500], 
                             'max_depth':[5, 10, 15]
                             }
                    },
              'lrm':{'model': LogisticRegression(solver='lbfgs', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 
                              'max_iter':[100, 500, 1000]
                             }
                    },
              'et':{'model': ExtraTreesClassifier(random_state=0), 
                    'params':{'n_estimators': [100, 200, 500], 
                              'max_depth': [5, 10, 15]
                             }
                    },
              'nn':{'model': MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', 
                                           power_t=0.5, max_iter=500, shuffle=True, tol=0.00001, verbose=True, 
                                           early_stopping=False, validation_fraction=0.1, random_state=0),
                    'params':{'hidden_layer_sizes': [(5, ), (10, ), (15, )],
                                 'learning_rate_init': [1e-1, 1e-2, 1e-3],
                                 'alpha':  [1e-2, 1e-3, 1e-4],
                                 'activation': ['tanh', 'relu', 'logistic']
                             }
                    }
             }
scores=[]
for model_name, mp in model_params.items():
    clf=GridSearchCV(mp['model'], mp['params'], cv = None, return_train_score = False, n_jobs = -1)
    clf.fit(X_train, y_train)
    scores.append({'model': model_name, 'best_score':clf.best_score_, 'best_params':clf.best_params_})

C:\Users\ambar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [121]:
# grid search scores
scores=pd.DataFrame(scores).sort_values(by = ['best_score'], ascending = False)
scores.to_csv('scores_root_end.csv')
scores

,model,best_score,best_params
3,et,0.729412,"{'max_depth': 5, 'n_estimators': 100}"
0,svm,0.720588,"{'C': 1000, 'gamma': 0.0001}"
1,rfm,0.720588,"{'max_depth': 10, 'n_estimators': 500}"
2,lrm,0.708824,"{'C': 100, 'max_iter': 100}"
4,nn,0.682353,"{'activation': 'relu', 'alpha': 0.01, 'hidden_..."


In [122]:
# models
scores = OrderedDict()
target_names = ['0', '1']

print('SUPPORT VECTOR MACHINES')
clf = SVC(random_state = 0, C = 1000, gamma = 0.0001, kernel = 'rbf')
clf.fit(X_train, y_train)
y_train_pred_svm = clf.predict(X_train)
y_test_pred_svm = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_svm)
mse_test = mean_squared_error(y_test, y_test_pred_svm)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_svm, target_names=target_names))
scores['SVC'] = [score, metrics.precision_score(y_test, y_test_pred_svm), metrics.recall_score(y_test, y_test_pred_svm),
                      metrics.f1_score(y_test, y_test_pred_svm), mse_test, mse_train]

print('##################################')
print('NEURAL NETWORKS')
clf = MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', power_t=0.5, max_iter=500, 
                    shuffle=True, tol=0.00001, verbose=True, 
                    early_stopping=False, validation_fraction=0.1,
                    hidden_layer_sizes=(5, ), activation='relu', alpha=0.01, 
                    learning_rate_init=0.1, random_state=0)
clf.fit(X_train, y_train)
y_train_pred_nn = clf.predict(X_train)
y_test_pred_nn = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_nn)
mse_test = mean_squared_error(y_test, y_test_pred_nn)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_nn, target_names=target_names))
scores['NN'] = [score, metrics.precision_score(y_test, y_test_pred_nn), metrics.recall_score(y_test, y_test_pred_nn),
                      metrics.f1_score(y_test, y_test_pred_nn), mse_test, mse_train]

print('##################################')
print('DECISION TREES')
clf = ExtraTreesClassifier(random_state=0, max_depth = 5, n_estimators = 100)
clf.fit(X_train, y_train)
y_train_pred_et = clf.predict(X_train)
y_test_pred_et = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_et)
mse_test = mean_squared_error(y_test, y_test_pred_et)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_et, target_names=target_names))
scores['ET'] = [score, metrics.precision_score(y_test, y_test_pred_et), 
                         metrics.recall_score(y_test, y_test_pred_et), 
                         metrics.f1_score(y_test, y_test_pred_et), mse_test, mse_train]

print('##################################')
print('LOGISTIC REGRESSION')
clf = LogisticRegression(solver='lbfgs', random_state = 0, C = 100, max_iter = 100)
clf.fit(X_train, y_train)
y_train_pred_lr = clf.predict(X_train)
y_test_pred_lr = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_lr)
mse_test = mean_squared_error(y_test, y_test_pred_lr)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_lr, target_names=target_names))
scores['LR'] = [score, metrics.precision_score(y_test, y_test_pred_lr), metrics.recall_score(y_test, y_test_pred_lr),
                      metrics.f1_score(y_test, y_test_pred_lr), mse_test, mse_train]

print('##################################')
print('RANDOM FOREST')
clf = RandomForestClassifier(random_state = 0, max_depth = 10, n_estimators = 500)
clf.fit(X_train, y_train)
y_train_pred_rf = clf.predict(X_train)
y_test_pred_rf = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_rf)
mse_test = mean_squared_error(y_test, y_test_pred_rf)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_rf, target_names=target_names))
scores['RF'] = [score, metrics.precision_score(y_test, y_test_pred_rf), metrics.recall_score(y_test, y_test_pred_rf),
                      metrics.f1_score(y_test, y_test_pred_rf), mse_test, mse_train]

SUPPORT VECTOR MACHINES
MSE Test: 0.313953488372093, MSE Train: 0.27058823529411763
Accuracy score: 0.686046511627907

              precision    recall  f1-score   support

           0       0.33      0.04      0.07        26
           1       0.70      0.97      0.81        60

    accuracy                           0.69        86
   macro avg       0.52      0.50      0.44        86
weighted avg       0.59      0.69      0.59        86

##################################
NEURAL NETWORKS
MSE Test: 0.313953488372093, MSE Train: 0.15294117647058825
Accuracy score: 0.686046511627907

              precision    recall  f1-score   support

           0       0.48      0.38      0.43        26
           1       0.75      0.82      0.78        60

    accuracy                           0.69        86
   macro avg       0.62      0.60      0.60        86
weighted avg       0.67      0.69      0.68        86

##################################
DECISION TREES
MSE Test: 0.3023255813953488, M

C:\Users\ambar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


MSE Test: 0.313953488372093, MSE Train: 0.008823529411764706
Accuracy score: 0.686046511627907

              precision    recall  f1-score   support

           0       0.44      0.15      0.23        26
           1       0.71      0.92      0.80        60

    accuracy                           0.69        86
   macro avg       0.58      0.54      0.52        86
weighted avg       0.63      0.69      0.63        86



In [123]:
# model scores
scores_df = pd.DataFrame.from_dict(scores, orient='index')
scores_df.columns=['accuracy', 'precision', 'recall', 'f1-score', 'mse test', 'mse train']
scores_df

,accuracy,precision,recall,f1-score,mse test,mse train
SVC,0.686047,0.698795,0.966667,0.811189,0.313953,0.270588
NN,0.686047,0.753846,0.816667,0.784000,0.313953,0.152941
ET,0.697674,0.702381,0.983333,0.819444,0.302326,0.261765
LR,0.674419,0.705128,0.916667,0.797101,0.325581,0.279412
RF,0.686047,0.714286,0.916667,0.802920,0.313953,0.008824


ANALYSIS ON TEST DATA

In [124]:
test_df['PLabel NN'] = y_test_pred_nn
test_df['PLabel ET'] = y_test_pred_et
test_df['PLabel RF'] = y_test_pred_rf

test_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = test_df.merge(test_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label 3'])
analysis['Minimum Time'] = analysis[['CUTS OFF_ROOT END', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Minimum Time,Minimum Nodes
0,neos-780889.mps.gz,176.675456,469.976354,1.0,1.0,0,1,1,176.675456,1.0
1,mzzv11.mps.gz,422.340278,1550.475830,1356.0,8880.0,1,1,1,422.340278,1356.0
2,chromaticindex32-8.mps.gz,1.025220,3.402795,1.0,249.0,1,1,0,1.025220,1.0
3,neos22.mps.gz,1.452381,18000.976538,1.0,4782054.0,0,1,1,1.452381,1.0
4,ab67-40-100.mps.gz,300.208641,331.249310,73.0,75.0,1,1,1,300.208641,73.0
...,...,...,...,...,...,...,...,...,...,...
81,mik-250-20-75-2.mps.gz,12.884775,35.434773,427.0,90650.0,1,1,1,12.884775,427.0
82,30_70_45_095_100.mps.gz,743.997969,246.544417,201.0,1.0,1,1,1,246.544417,1.0
83,neos-4322846-ryton.mps.gz,818.066564,905.172764,664.0,522.0,0,1,1,818.066564,522.0
84,ran14x18-disj-8.mps.gz,1658.803667,16650.042406,312024.0,14591597.0,1,1,1,1658.803667,312024.0


In [125]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == 0.0:
        rf_pred_time.append(row['CUTS OFF_ROOT END'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == 0.0:
        nn_pred_time.append(row['CUTS OFF_ROOT END'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == 0.0:
        et_pred_time.append(row['CUTS OFF_ROOT END'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,neos-780889.mps.gz,176.675456,469.976354,1.0,1.0,0,1,1,176.675456,1.0,176.675456,469.976354,176.675456,1.0,1.0,1.0
1,mzzv11.mps.gz,422.340278,1550.475830,1356.0,8880.0,1,1,1,422.340278,1356.0,422.340278,422.340278,422.340278,1356.0,1356.0,1356.0
2,chromaticindex32-8.mps.gz,1.025220,3.402795,1.0,249.0,1,1,0,1.025220,1.0,3.402795,1.025220,1.025220,249.0,1.0,1.0
3,neos22.mps.gz,1.452381,18000.976538,1.0,4782054.0,0,1,1,1.452381,1.0,1.452381,18000.976538,1.452381,1.0,4782054.0,1.0
4,ab67-40-100.mps.gz,300.208641,331.249310,73.0,75.0,1,1,1,300.208641,73.0,300.208641,300.208641,300.208641,73.0,73.0,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,mik-250-20-75-2.mps.gz,12.884775,35.434773,427.0,90650.0,1,1,1,12.884775,427.0,12.884775,12.884775,12.884775,427.0,427.0,427.0
82,30_70_45_095_100.mps.gz,743.997969,246.544417,201.0,1.0,1,1,1,246.544417,1.0,743.997969,743.997969,743.997969,201.0,201.0,201.0
83,neos-4322846-ryton.mps.gz,818.066564,905.172764,664.0,522.0,0,1,1,818.066564,522.0,818.066564,905.172764,818.066564,664.0,522.0,664.0
84,ran14x18-disj-8.mps.gz,1658.803667,16650.042406,312024.0,14591597.0,1,1,1,1658.803667,312024.0,1658.803667,1658.803667,1658.803667,312024.0,312024.0,312024.0


In [126]:
testing = analysis.copy()

In [127]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [128]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [129]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [130]:
SHIFT_TIME = 60
SHIFT_NODES = 1000

df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_ROOT END'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [131]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                           Time                             \
Setting                              Cut        No Cut      Perfect   
Metric         Bucket                                                 
Geometric Mean [0,18000)      319.066487    498.651744   248.558959   
               [10,18000)     668.496297   1033.765256   491.380974   
               [100,18000)   1466.016029   1684.176252  1001.969474   
               [1000,18000)  7474.026746   5668.473076  4000.629324   
               [3600,18000)  9273.146949  12143.732338  7847.582582   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      313.477382   317.027359   335.763557   
               [10,18000)     653.436073   663.152040   639.857061   
               [100,18000)   1439.236611  1467.898665  1397.185152   
               [1000,18000)  6768.850643  7474.026746  6006.940480   
               [3600,18000)  9199.094521  9273.146949  8693.179270   

Parameter                             Node                                \
Setting                                Cut         No Cut        Perfect   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6490.303382   18481.916810    5701.479095   
               [10,18000)     13336.715273   39828.740403   11324.046990   
               [100,18000)    23928.740189   42297.814134   19726.728761   
               [1000,18000)  247235.401228  319102.176897  165249.332318   
               [3600,18000)  530800.984581  819106.252775  482884.718216   

Parameter                                                                 \
Setting                                 RF             ET             NN   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6409.196286    6447.234089    7640.448502   
               [10,18000)     13075.677167   13224.301569   13619.053396   
               [100,18000)    23623.773963   23988.542792   24894.013459   
               [1000,18000)  226645.089800  247235.401228  243337.288100   
               [3600,18000)  529423.905324  530800.984581  646928.277282   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)           86  
               [10,18000)          63  
               [100,18000)         45  
               [1000,18000)        19  
               [3600,18000)         9

In [132]:
# we use RF for this analysis
imp_time_rf = []
imp_nodes_rf = []

for i, j in zip(t_rf, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_rf.append(imp)

for i, j in zip(n_rf, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_rf.append(imp)

In [133]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_rf
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_rf
avg_bb_df

Parameter                           Time                             \
Setting                              Cut        No Cut      Perfect   
Metric         Bucket                                                 
Geometric Mean [0,18000)      319.066487    498.651744   248.558959   
               [10,18000)     668.496297   1033.765256   491.380974   
               [100,18000)   1466.016029   1684.176252  1001.969474   
               [1000,18000)  7474.026746   5668.473076  4000.629324   
               [3600,18000)  9273.146949  12143.732338  7847.582582   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      313.477382   317.027359   335.763557   
               [10,18000)     653.436073   663.152040   639.857061   
               [100,18000)   1439.236611  1467.898665  1397.185152   
               [1000,18000)  6768.850643  7474.026746  6006.940480   
               [3600,18000)  9199.094521  9273.146949  8693.179270   

Parameter                             Node                                \
Setting                                Cut         No Cut        Perfect   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6490.303382   18481.916810    5701.479095   
               [10,18000)     13336.715273   39828.740403   11324.046990   
               [100,18000)    23928.740189   42297.814134   19726.728761   
               [1000,18000)  247235.401228  319102.176897  165249.332318   
               [3600,18000)  530800.984581  819106.252775  482884.718216   

Parameter                                                                 \
Setting                                 RF             ET             NN   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6409.196286    6447.234089    7640.448502   
               [10,18000)     13075.677167   13224.301569   13619.053396   
               [100,18000)    23623.773963   23988.542792   24894.013459   
               [1000,18000)  226645.089800  247235.401228  243337.288100   
               [3600,18000)  529423.905324  530800.984581  646928.277282   

Parameter                   Instances Improvement (%)            
Setting                                          Time     Nodes  
Metric         Bucket                                            
Geometric Mean [0,18000)           86       -1.751706 -1.249666  
               [10,18000)          63       -2.252851 -1.957289  
               [100,18000)         45       -1.826680 -1.274477  
               [1000,18000)        19       -9.435022 -8.328221  
               [3600,18000)         9       -0.798568 -0.259434

ANALYSIS ON TRAINING DATA

In [134]:
train_df

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,TIME FIRST ROUND,TIME ROOT END,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,ROUND OF CUTS,OBJECTIVE VALUE ROOT NODE,vars,int_vars,constr,linsetpart_constr,linsetpack_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linintknap_constr,linmixbin_constr,nonzeroes,dynamism_mean,presol_vars,presol_int_vars,presol_constr
255,neos-5140963-mincio.mps.gz,16296.812996,20739.942829,0.027525,2829.124553,5624805.0,5527433.2,8.877000e+03,8.877000e+03,8877.000000,29.0,1.059800e+04,5.278115,0.000000,5.214936,0.152174,0.000000,0.000000,0.0,0.0,0.000000,0.847826,0.023126,0.971717,5.278115,0.000000,5.214936
313,neos8.mps.gz,16.780323,72.893637,16.406966,16.410093,1.0,1.0,-4.390700e+04,-4.390700e+04,-3719.000000,1.0,-3.719000e+03,10.053114,0.000172,10.743415,0.000000,0.000086,0.000259,0.0,0.0,0.000000,0.497884,0.000291,0.248909,10.048280,0.000173,10.740995
229,neos-4391920-timok.mps.gz,299.384161,868.639419,556.933012,578.506055,1.0,1.0,2.282219e-03,2.282219e-03,0.002282,111.0,5.451422e-03,11.449410,0.000000,12.125811,0.000336,0.000000,0.000000,0.0,0.0,0.000000,0.000623,0.000034,1.150550,11.449410,0.000000,12.125811
5,30_70_45_095_98.mps.gz,76.557754,96.309816,13.898364,69.983411,1.0,1.0,-5.967000e+03,-5.967000e+03,11.500000,11.0,1.200000e+01,9.304741,0.000000,9.431161,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.160532,0.000338,0.000000,9.303649,0.000000,9.430359
425,wachplan.mps.gz,1434.605408,1873.745528,2.617340,97.206743,67520.0,98056.0,-2.400000e+01,-2.400000e+01,-9.000000,12.0,-9.000000e+00,8.119994,0.000298,7.347944,0.000000,0.000000,0.144237,0.0,0.0,0.000000,0.018030,0.017120,0.000000,7.741534,0.000434,6.582025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,ns1644855.mps.gz,860.987882,853.974045,0.000000,0.000000,1.0,1.0,-2.000000e+05,-2.000000e+05,0.000000,0.0,-1.524333e+03,10.315597,0.000000,10.613934,0.004914,0.000000,0.000000,0.0,0.0,0.000000,0.493882,0.001717,0.642555,10.315564,0.000000,10.328690
192,neos-3065804-namu.mps.gz,11.837614,11.855551,0.000000,0.000000,1.0,1.0,-3.689729e+08,-3.689729e+08,0.000000,0.0,-3.689729e+08,8.464003,0.000000,8.095599,0.000000,0.111280,0.000000,0.0,0.0,0.000000,0.666768,0.036965,1.758950,8.464003,0.000000,8.095599
117,ic97_tension.mps.gz,20.023839,37.568637,0.067639,5.569328,22278.0,83938.0,2.606000e+03,2.606000e+03,3900.698555,40.0,3.936000e+03,6.555357,0.005690,5.765191,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.987461,0.009230,1.003350,6.481577,0.006126,5.736572
47,bppc8-02.mps.gz,0.677820,0.464540,0.052751,0.347415,125.0,177.0,3.600000e+01,3.600000e+01,487.000000,26.0,4.870000e+02,5.446737,0.004310,4.077537,0.288136,0.000000,0.000000,0.0,0.0,0.016949,0.559322,0.320500,1.311860,5.438079,0.000000,4.060443


In [135]:
train_df['PLabel NN'] = y_train_pred_nn
train_df['PLabel ET'] = y_train_pred_et
train_df['PLabel RF'] = y_train_pred_rf

train_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = train_df.merge(train_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_ROOT END', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label'])
analysis['Minimum Time'] = analysis[['CUTS OFF_ROOT END', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes
0,neos-5140963-mincio.mps.gz,16296.812996,20739.942829,5624805.0,5527433.2,1,1,1,1,16296.812996,5527433.2
1,neos8.mps.gz,16.780323,72.893637,1.0,1.0,1,1,1,1,16.780323,1.0
2,neos-4391920-timok.mps.gz,299.384161,868.639419,1.0,1.0,1,1,1,1,299.384161,1.0
3,30_70_45_095_98.mps.gz,76.557754,96.309816,1.0,1.0,1,1,1,1,76.557754,1.0
4,wachplan.mps.gz,1434.605408,1873.745528,67520.0,98056.0,1,1,1,1,1434.605408,67520.0
...,...,...,...,...,...,...,...,...,...,...,...
335,ns1644855.mps.gz,860.987882,853.974045,1.0,1.0,1,1,0,0,853.974045,1.0
336,neos-3065804-namu.mps.gz,11.837614,11.855551,1.0,1.0,1,1,1,1,11.837614,1.0
337,ic97_tension.mps.gz,20.023839,37.568637,22278.0,83938.0,1,1,1,1,20.023839,22278.0
338,bppc8-02.mps.gz,0.677820,0.464540,125.0,177.0,0,1,0,0,0.464540,125.0


In [136]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == 0.0:
        rf_pred_time.append(row['CUTS OFF_ROOT END'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == 0.0:
        nn_pred_time.append(row['CUTS OFF_ROOT END'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == 0.0:
        et_pred_time.append(row['CUTS OFF_ROOT END'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,CUTS OFF_ROOT END,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,neos-5140963-mincio.mps.gz,16296.812996,20739.942829,5624805.0,5527433.2,1,1,1,1,16296.812996,5527433.2,16296.812996,16296.812996,16296.812996,5624805.0,5624805.0,5624805.0
1,neos8.mps.gz,16.780323,72.893637,1.0,1.0,1,1,1,1,16.780323,1.0,16.780323,16.780323,16.780323,1.0,1.0,1.0
2,neos-4391920-timok.mps.gz,299.384161,868.639419,1.0,1.0,1,1,1,1,299.384161,1.0,299.384161,299.384161,299.384161,1.0,1.0,1.0
3,30_70_45_095_98.mps.gz,76.557754,96.309816,1.0,1.0,1,1,1,1,76.557754,1.0,76.557754,76.557754,76.557754,1.0,1.0,1.0
4,wachplan.mps.gz,1434.605408,1873.745528,67520.0,98056.0,1,1,1,1,1434.605408,67520.0,1434.605408,1434.605408,1434.605408,67520.0,67520.0,67520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,ns1644855.mps.gz,860.987882,853.974045,1.0,1.0,1,1,0,0,853.974045,1.0,853.974045,860.987882,860.987882,1.0,1.0,1.0
336,neos-3065804-namu.mps.gz,11.837614,11.855551,1.0,1.0,1,1,1,1,11.837614,1.0,11.837614,11.837614,11.837614,1.0,1.0,1.0
337,ic97_tension.mps.gz,20.023839,37.568637,22278.0,83938.0,1,1,1,1,20.023839,22278.0,20.023839,20.023839,20.023839,22278.0,22278.0,22278.0
338,bppc8-02.mps.gz,0.677820,0.464540,125.0,177.0,0,1,0,0,0.464540,125.0,0.464540,0.464540,0.677820,177.0,177.0,125.0


In [137]:
training = analysis.copy()

In [138]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [139]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [140]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [141]:
SHIFT_TIME = 60
SHIFT_NODES = 1000

df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_ROOT END'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [142]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                            Time                             \
Setting                               Cut        No Cut      Perfect   
Metric         Bucket                                                  
Geometric Mean [0,18000)       350.004190    635.391290   308.855793   
               [10,18000)      599.675183   1102.555406   518.405997   
               [100,18000)    1446.376544   2532.147809  1204.070491   
               [1000,18000)   5590.631431   8522.819730  4515.145540   
               [3600,18000)  11061.303018  14849.022167  9549.536313   

Parameter                                                             \
Setting                               RF            ET            NN   
Metric         Bucket                                                  
Geometric Mean [0,18000)      309.822313    341.240225    337.662967   
               [10,18000)     520.308132    582.032593    566.699264   
               [100,18000)   1209.853430   1394.472081   1355.646776   
               [1000,18000)  4557.679534   5443.356337   5064.119084   
               [3600,18000)  9709.523171  10722.683372  10369.766706   

Parameter                            Node                              \
Setting                               Cut        No Cut       Perfect   
Metric         Bucket                                                   
Geometric Mean [0,18000)      4096.962302  13334.591257   3569.851283   
               [10,18000)     6523.995528  20540.982619   5565.633702   
               [100,18000)   12029.019777  33609.857233   9892.687234   
               [1000,18000)  31303.902584  80146.933111  24601.733662   
               [3600,18000)  42117.500475  99240.510010  32976.658275   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)      3848.307176   4002.822206   4239.484702   
               [10,18000)     6057.293499   6350.057611   6568.078121   
               [100,18000)   10998.820502  11597.565901  12258.988279   
               [1000,18000)  28404.184997  30909.497921  30421.808953   
               [3600,18000)  40894.068555  41440.213661  45414.377366   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)          340  
               [10,18000)         271  
               [100,18000)        189  
               [1000,18000)        91  
               [3600,18000)        51

In [143]:
# We use RF for our analysis
imp_time_rf = []
imp_nodes_rf = []

for i, j in zip(t_rf, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_rf.append(imp)

for i, j in zip(n_rf, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_rf.append(imp)

In [144]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_rf
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_rf
avg_bb_df

Parameter                            Time                             \
Setting                               Cut        No Cut      Perfect   
Metric         Bucket                                                  
Geometric Mean [0,18000)       350.004190    635.391290   308.855793   
               [10,18000)      599.675183   1102.555406   518.405997   
               [100,18000)    1446.376544   2532.147809  1204.070491   
               [1000,18000)   5590.631431   8522.819730  4515.145540   
               [3600,18000)  11061.303018  14849.022167  9549.536313   

Parameter                                                             \
Setting                               RF            ET            NN   
Metric         Bucket                                                  
Geometric Mean [0,18000)      309.822313    341.240225    337.662967   
               [10,18000)     520.308132    582.032593    566.699264   
               [100,18000)   1209.853430   1394.472081   1355.646776   
               [1000,18000)  4557.679534   5443.356337   5064.119084   
               [3600,18000)  9709.523171  10722.683372  10369.766706   

Parameter                            Node                              \
Setting                               Cut        No Cut       Perfect   
Metric         Bucket                                                   
Geometric Mean [0,18000)      4096.962302  13334.591257   3569.851283   
               [10,18000)     6523.995528  20540.982619   5565.633702   
               [100,18000)   12029.019777  33609.857233   9892.687234   
               [1000,18000)  31303.902584  80146.933111  24601.733662   
               [3600,18000)  42117.500475  99240.510010  32976.658275   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)      3848.307176   4002.822206   4239.484702   
               [10,18000)     6057.293499   6350.057611   6568.078121   
               [100,18000)   10998.820502  11597.565901  12258.988279   
               [1000,18000)  28404.184997  30909.497921  30421.808953   
               [3600,18000)  40894.068555  41440.213661  45414.377366   

Parameter                   Instances Improvement (%)            
Setting                                          Time     Nodes  
Metric         Bucket                                            
Geometric Mean [0,18000)          340      -11.480399 -6.069256  
               [10,18000)         271      -13.235007 -7.153623  
               [100,18000)        189      -16.352803 -8.564283  
               [1000,18000)        91      -18.476480 -9.263118  
               [3600,18000)        51      -12.220801 -2.904807

COMPARITIVE ANALYSIS OF TRAIN AND TEST DATA

In [146]:
df = [training, testing]

t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_ROOT END'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [147]:
train_test_classes = ['Geometric Mean']
train_test_sets = ['Train Set', 'Test Set']

In [148]:
train_test_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut', 'No Cut','Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

train_test_row_names = pd.MultiIndex.from_product(
    [train_test_classes, train_test_sets],
    names=['Metric', 'Set'])

train_test_df = pd.DataFrame(
    index = train_test_row_names,
    dtype = float)

train_test_df = pd.DataFrame(
    columns = train_test_cols,
    index = train_test_row_names,
    dtype = float)

In [149]:
train_test_df['Instances'] = instances

train_test_df[('Time', 'Cut')] = t_cuts
train_test_df[('Time', 'No Cut')] = t_nocuts
train_test_df[('Time', 'RF')] = t_rf
train_test_df[('Time', 'ET')] = t_et
train_test_df[('Time', 'NN')] = t_nn

train_test_df[('Time', 'Perfect')] = t_perfect

train_test_df[('Node', 'Cut')] = n_cuts
train_test_df[('Node', 'No Cut')] = n_nocuts
train_test_df[('Node', 'RF')] = n_rf
train_test_df[('Node', 'ET')] = n_et
train_test_df[('Node', 'NN')] = n_nn

train_test_df[('Node', 'Perfect')] = n_perfect

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  350.004190  635.391290  308.855793  309.822313   
               Test Set   319.066487  498.651744  248.558959  313.477382   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  341.240225  337.662967  4096.962302  13334.591257   
               Test Set   317.027359  335.763557  6490.303382  18481.916810   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  3569.851283  3848.307176  4002.822206  4239.484702   
               Test Set   5701.479095  6409.196286  6447.234089  7640.448502   

Parameter                Instances  
Setting                             
Metric         Set                  
Geometric Mean Train Set       340  
               Test Set         86

In [150]:
## We use the best ml strategy for this analysis
best_time = []
best_nodes = []

zip_time = zip(t_rf, t_et, t_nn)
zip_time = list(zip_time)

zip_nodes = zip(n_rf, n_et, n_nn)
zip_nodes = list(zip_nodes)

for i in zip_time:
    best_time.append(min(i))

for i in zip_nodes:
    best_nodes.append(min(i))

In [151]:
imp_time = []
imp_nodes = []

for i, j in zip(best_time, t_cuts):
    imp = ((i - j)/j) * 100
    imp_time.append(imp)
    
for i, j in zip(best_nodes, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes.append(imp)

In [152]:
train_test_df[('Improvement (%)', 'Time')] = imp_time
train_test_df[('Improvement (%)', 'Nodes')] = imp_nodes

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  350.004190  635.391290  308.855793  309.822313   
               Test Set   319.066487  498.651744  248.558959  313.477382   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  341.240225  337.662967  4096.962302  13334.591257   
               Test Set   317.027359  335.763557  6490.303382  18481.916810   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  3569.851283  3848.307176  4002.822206  4239.484702   
               Test Set   5701.479095  6409.196286  6447.234089  7640.448502   

Parameter                Instances Improvement (%)            
Setting                                       Time     Nodes  
Metric         Set                                            
Geometric Mean Train Set       340      -11.480399 -6.069256  
               Test Set         86       -1.751706 -1.249666

In [153]:
# data for different notebook
for_plot = pd.concat([training, testing])
for_plot = for_plot.reset_index()
# for_plot.to_csv('root_end_plot.csv')
# testing.to_csv('testing_plot.csv')
# training.to_csv('training_plot.csv')